# This aims to generate 3D Grad-CAM for regression networks, or, RAM_3d

In [5]:
import os
import sys

# sys.path.append("../..")
sys.path.append("../../..")

import matplotlib.pyplot as plt
import torch
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from tqdm import tqdm
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import numpy as np
from tqdm import tqdm
from mlflow import log_metric, log_metrics, log_param, log_params
import mlflow
from mlflow.tracking import MlflowClient
from medcam3d import GradCAM

from lung_function.modules.path import PFTPath
from lung_function.modules.datasets import all_loaders
# from lung_function.modules.cam import GradCAM
from lung_function.scripts.run import Run
from lung_function.modules.set_args import get_args


The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
usage: ipykernel_launcher.py [-h]
                             [--net {vgg11_3d,vit3,vgg16_3d,vgg19_3d,r3d_resnet,cnn3fc1,cnn4fc2,cnn5fc2,cnn6fc2,cnn2fc1,cnn3fc2,r3d_18,slow_r50,slowfast_r50,x3d_xs,x3d_s,x3d_m,x3d_l,pointnet_reg,pointnet2_reg,vgg11_3d,pointnext}]
                             [--cfg CFG] [--fc2_nodes FC2_NODES]
                             [--fc1_nodes FC1_NODES]
                             [--pointnet_fc_ls POINTNET_FC_LS]
                             [--dp_fc1_flag DP_FC1_FLAG]
                             [--npoint_base NPOINT_BASE]
                             [--radius_base RADIUS_BASE]
                             [--nsample_base NSAMPLE_BASE] [--width WIDTH]

SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [2]:
class Args:  # convert dict to class attribute
    def __init__(self, d=None):
        if d is not None:
            for key, value in d.items():
                if value == 'True':
                    value = True
                elif value == 'False':
                    value = False
                else:
                    try:
                        value = float(value)  # convert to float value if possible
                        try:
                            if int(value) == value:  # convert to int if possible
                                value = int(value)
                        except Exception:
                            pass
                    except Exception:
                        pass
                setattr(self, key, value)

In [3]:
class RegressionOutputTarget:  # devine my custom target
    def __init__(self, name):
        self.name = name  # possible names: DLCOc, FEV1, FVC, TLC, DLCOcPP, FEV1PP, FVCPP, TLCPP

    def __call__(self, model_output):
        if len(model_output.shape) == 1:  # shape: (N,), A vector
            if model_output.shape[0] == 1:
                return model_output  # TODO: for the single output network
            return model_output[self.category]  # return one node
        
        # shape: (M, N), A 2d array
        return model_output[:, self.category]  # return a vector 


In [4]:
# update parameters
AttentionMethod = "GradCAM"  # or others
Ex_id = 2656  # FVC, fold1

In [7]:
# retrive the run for the ex id
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")
client = MlflowClient()
run_ls = client.search_runs(experiment_ids=[experiment.experiment_id],
                            filter_string=f"params.id LIKE '%{Ex_id}%'")
run_ = run_ls[0]
args_dt = run_.data.params  # extract the hyper parameters

NameError: name 'mlflow' is not defined

In [ ]:


args = Args(args_dt)  #convert to object
args.workers=1

args.use_cuda = True
device = torch.device("cuda:0" if args.use_cuda else "cpu")

    
ckpt = torch.load(self.mypath.model_fpath, map_location=device)
if type(ckpt) is dict and 'model' in ckpt:
    model = ckpt['model']
else:
    model = ckpt
self.net.load_state_dict(model)  # model_fpath need to exist

target_layers = [self.net.layer4[-1]]  # TODO: change this line select which layer
targets = [RegressionOutputTarget(args.target)]  # TODO: change it to select which output
if AttentionMethod=="GradCAM":
    cam = GradCAM(model=self.net, target_layers=target_layers, use_cuda=args.use_cuda)
    # attention = GradCAM(Ex_id, args_dt, 'last_maxpool')
else:
    raise Exception(f"Please set the correct AttentionMethod")



mypath = PFTPath(Ex_id, check_id_dir=False, space=args.ct_sp)

# select the top accurate patients
mode = 'valid'
label_all = pd.read_csv(mypath.save_label_fpath(mode))
pred_all = pd.read_csv(mypath.save_pred_fpath(mode))
mae_all = (label_all - pred_all).abs()
mae_all['average'] = mae_all.mean(numeric_only=True, axis=1)
label_all_sorted = label_all.loc[mae_all['average'].argsort()[:max_img_nb]]
top_pats = label_all_sorted['pat_id'].to_list()

data_dt = all_loaders(mypath.data_dir, mypath.label_fpath, args, datasetmode='valid', top_pats=top_pats)
dataloader = data_dt['valid']

for data in dataloader:
    batch_pat_id = data['pat_id'].detach().numpy()
    batch_x = data['image']
    batch_y = data['label']
    batch_ori = data['origin'].detach().numpy()
    batch_sp = data['spacing'].detach().numpy()

    for pat_id, image, ori, sp, label in zip(batch_pat_id, batch_x, batch_ori, batch_sp, batch_y):
        img = image[None].to(device)
        print(img.shape)
        # attention.run(pat_id, image, ori, sp, label)  # for my own code of RAM
        print('Finish pat_id: ', pat_id)
print("Finish all")

